<a href="https://colab.research.google.com/github/Kiritoslove/CIVE_70019_70057/blob/main/notebooks/coursework_2025_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework: Hydraulic model calibration


### CIVE 70019/70057
Department of Civil and Environmental Engineering, Imperial College London

You have been tasked with the evaluation and calibration of the hydraulic model of EXNING, a district metered area (DMA) in Anglian Water's (AW) water distribution network. To achieve this, you have been provided:
* a recently calibrated hydraulic model of the EXNING DMA (2019),
* loading conditions (demands, reservoir heads) and head measurements covering a period of 4 days.

The objective of the coursework is to prepare a short calibration report for AW by completing the tasks below and answering the questions based on your results.

You have been provided the following information about EXNING:
* EXNING is part of a larger system of cascading DMAs: EXNING is fed by the NEWSEV DMA and feeds into the BURWEL DMA.
* The "reservoir" head and total demand of EXNING are derived from flow and pressure sensors at the DMA inlet (& outlet).
* Node elevations have been updated in the provided model following a GPS survey of sensor locations.

First, we must clone the GitHub repository and install dependencies (only run this once).

In [26]:
# run this cell once
import sys
import os

if 'google.colab' in sys.modules:
  !git clone https://github.com/bradleywjenks/CIVE_70019_70057.git
  !pip install wntr==1.2
  !pip install cvxpy
  !apt-get install libsuitesparse-dev && pip install scikit-sparse

fatal: destination path 'CIVE_70019_70057' already exists and is not an empty directory.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsuitesparse-dev is already the newest version (1:5.10.1+dfsg-4build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [27]:
# load packages
import numpy as np
from numpy import linalg as la
import networkx as nx
import pandas as pd
import wntr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import copy
from datetime import datetime, timedelta
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

# improve matplotlib image quality
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

ModuleNotFoundError: No module named 'numpy.char'

### Load network properties and operational data

Load functions created in previous assignments.

In [ ]:
# load functions from src folder
if 'google.colab' in sys.modules:
    sys.path.append('/content/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *
else:
    sys.path.append('/home/bradw/workspace/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *

Load network .inp file and operational data from the module repository's data directory.

In [ ]:
if 'google.colab' in sys.modules:
    # if run in Google Colab
    data_dir = '/content/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/content/CIVE_70019_70057/data/networks/'
else:
    # replace with local directory
    data_dir = '/home/bradw/workspace/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/home/bradw/workspace/CIVE_70019_70057/data/networks/'

net_name = 'exning.inp'
data_name = 'exning_coursework_dataset.npy'

# load operational data
data = np.load(os.path.join(data_dir, data_name), allow_pickle=True).item()
h_data = data['h_data']
sensor_idx = data['sensor_idx'] - 1 # matlab to python index offset
d_data = data['d']
h0_data = data['h0'].reshape(-1, 1).T

# load network properties
wdn = load_network_data(os.path.join(net_dir, net_name))
A12 = wdn.A12
A10 = wdn.A10
A21 = A12.T
net_info = wdn.net_info
link_df = wdn.link_df
node_df = wdn.node_df
demand_df = wdn.demand_df

In [ ]:
print(d_data.shape)

Plot sensor nodes in network. We provide a plotting function below.

In [ ]:
#### DO NOT CHANGE THIS ####
# define plotting function using networkx
def plot_network(wdn, sensor_idx, vals=None, highlight_valves=None):

    # create figure and axis
    fig, ax = plt.subplots(figsize=(10, 7))

    # unload data
    link_df = wdn.link_df
    node_df = wdn.node_df
    net_info = wdn.net_info

    # draw network
    uG = nx.from_pandas_edgelist(link_df, source='node_out', target='node_in')
    pos = {row['node_ID']: (row['xcoord'], row['ycoord']) for _, row in node_df.iterrows()}
    nx.draw(uG, pos, ax=ax, node_size=20, node_shape='o')

    # draw reservoir
    nx.draw_networkx_nodes(
        uG, pos, nodelist=net_info['reservoir_names'],
        node_size=100, node_shape='s', node_color='black', ax=ax
    )

    # draw sensor nodes
    sensor_names = [net_info['junction_names'][i] for i in sensor_idx]
    nx.draw_networkx_nodes(
        uG, pos, nodelist=sensor_names, node_size=100,
        node_shape='o', node_color='red', edgecolors='white', ax=ax
    )

    # reservoir labels
    reservoir_labels = {node: 'Reservoir' for node in net_info['reservoir_names']}
    labels_res = nx.draw_networkx_labels(uG, pos, reservoir_labels, font_size=12, ax=ax)
    for _, label in labels_res.items():
        label.set_y(label.get_position()[1] + 1750)

    # sensor labels
    sensor_labels = {node: str(idx+1) for (idx, node) in enumerate(sensor_names)}
    labels_sen = nx.draw_networkx_labels(uG, pos, sensor_labels, font_size=12, ax=ax)
    for _, label in labels_sen.items():
        label.set_y(label.get_position()[1] + 1750)

    # plot sensor values if provided
    if vals is not None:
        cmap = cm.get_cmap('RdYlGn_r')
        nx.draw_networkx_nodes(
            uG, pos, nodelist=sensor_names, node_size=100,
            node_shape='o', node_color=vals, cmap=cmap, edgecolors='white', ax=ax
        )

        # create color bar
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array(vals)
        colorbar = plt.colorbar(sm, ax=ax)
        colorbar.set_label('Mean pressure residual [m]', fontsize=12)

    # highlight link
    if highlight_valves is not None:
        nx.draw_networkx_nodes(uG, pos, highlight_valves, node_size=200, node_shape='d', node_color='limegreen', edgecolors='white')

    # display plot
    plt.show()

Visualise the EXNING network graph and highlight information relevant to the evaluation of the hydraulic model.

In [ ]:
plot_network(wdn, sensor_idx)

Simulate initial (uncalibrated) network hydraulics over 4 days. We first define a function to solve network hydraulics using the `wntr` package, which we used previously in the hyraulic modelling notebook. The following tasks are performed in this function:
- Load network properties
- Modify simulation time to match operational data
- Assign h0 data at model reservoir
- Scale and apply new demand pattern from inflow data
- Option to modify pipe roughness (or HW) coefficients

<font color="blue">NB: this is done for you below. Provided the correct inputs, you do not need to replicate this function.

In [ ]:
#### DO NOT CHANGE THIS ####
# hydraulic solver function
def hydraulic_solver(inp_file, d_data, h0_data, C=None, demand=False):

    # load network from wntr
    inp_file = os.path.join(net_dir, net_name)
    wn = wntr.network.WaterNetworkModel(inp_file)

    # get network properties
    reservoir_names = wn.reservoir_name_list
    junction_names = wn.junction_name_list
    link_names = wn.link_name_list

    # modify simulation time and hydraulic time step
    nt = h0_data.shape[1]
    wn.options.time.duration = (nt - 1) * 3600
    wn.options.time.hydraulic_timestep = 3600
    wn.options.time.pattern_timestep = 3600
    wn.options.time.report_timestep = 3600

    # assign reservoir data
    for (i, name) in enumerate(reservoir_names):
        wn.add_pattern(f'h0_{i}', h0_data[i])
        reservoir = wn.get_node(name)
        reservoir.head_timeseries.base_value = 1
        reservoir.head_timeseries.pattern_name = wn.get_pattern(f'h0_{i}')

    # replace demand data
    for idx, name in enumerate(junction_names):

        if any(val != 0 for val in d_data[idx, :]):
            node = wn.get_node(name)
            d_pat = d_data[idx, :]
            wn.add_pattern('d_'+name, d_pat)

            for (i, num) in enumerate(node.demand_timeseries_list):
                if i == 0:
                    node.demand_timeseries_list[i].base_value = 1
                    node.demand_timeseries_list[i].pattern_name = 'd_'+name
                else:
                    node.demand_timeseries_list[i].base_value = None
                    node.demand_timeseries_list[i].pattern_name = None

    # assign roughness (or HW) coefficients
    if C is not None:
        for name, link in wn.links():

            # check if the link is a pipe
            if isinstance(link, wntr.network.Pipe):
                link.roughness = C[link_names.index(name)]

            # check if link is a valve
            elif isinstance(link, wntr.network.Valve):
                link.minor_loss = C[link_names.index(name)]
                link.initial_setting = C[link_names.index(name)]

    # run simulation and get results
    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()

    q_sim = results.link['flowrate'].T
    h_sim = results.node['head'].T
    h_sim = h_sim[~h_sim.index.isin(reservoir_names)] # delete reservoir nodes
    d = results.node['demand'].T
    d = d[~d.index.isin(reservoir_names)] # delete reservoir nodes


    if demand == True:
        return d.to_numpy()
    else:
        return q_sim.to_numpy(), h_sim.to_numpy()

### Part 0: Preliminary evaluation

Run simulation with initial $C_0$ values.

In [ ]:
C_0 = link_df['C'].to_numpy()
_, h_0 = hydraulic_solver(wdn, d_data, h0_data,C=C_0)

Compare simulated heads at sensor nodes (node indices in `sensor_idx`) with the simulated heads over the 4-day period. Visualise the results with, e.g. a boxplot of pressure residuals on the test dataset.

In [ ]:
h_meas = h_data  # true observation data
h_sim = h_0[sensor_idx,:]  # h by simulation


h_residuals = h_meas - h_sim
h_residuals_square = np.square(h_meas - h_sim)

print("Residuals shape:", h_residuals.shape)
mse = np.mean((h_0[sensor_idx,: ] - data['h_data']) ** 2)
print(f"\n总均方误差 (MSE): {mse:.6f}")

In [ ]:
# compute pressure residuals
residuals_0 = h_0[sensor_idx,:] - h_data

# create figure and axis for box plots
fig, ax = plt.subplots()
boxplot = ax.boxplot(residuals_0.T, medianprops=dict(color="red", linewidth=1.0), flierprops=dict(marker="+", markeredgecolor="red"), whiskerprops=dict(linestyle=(5, (8, 5)), linewidth=0.6), boxprops=dict(color='black', linewidth=0.6), capprops=dict(linewidth=0.6))
ax.set_xlabel('Sensor index', fontsize=12)
ax.set_ylabel('Pressure residual [m]', fontsize=12)

In [ ]:
# spatial residuals plot
mean_residuals_0 = np.mean(residuals_0, axis=1)
plot_network(wdn, sensor_idx, vals=mean_residuals_0)

In [ ]:
# C_0 value scatter plot (valves only)
valve_C = [C_0[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
fig, ax = plt.subplots()
ax.scatter(range(len(valve_C)), valve_C, facecolors='none', edgecolors='b',linewidths=0.75)
ax.set_xlabel('Valve index', fontsize=12)
ax.set_ylabel('Initial C_0 value', fontsize=12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

In [ ]:
mean_residuals = np.mean(h_residuals, axis=1)
plot_network(wdn, sensor_idx, vals=mean_residuals)

In [ ]:
import seaborn as sns


#print(h_residuals)
plt.figure(figsize=(10, 6))

# seaborn boxplot
sns.boxplot(data=h_residuals.T)
plt.xticks(ticks=range(14), labels=range(1, 15))
plt.title("Boxplot of Residuals per sensor")
plt.xlabel("sensor_index")
plt.ylabel("Residuals")

In [ ]:
plt.figure(figsize=(12, 6))
time_points = np.arange(h_0.shape[1])  # Create a time index

sensor_labels = {sensor: str(idx+1) for idx, sensor in enumerate(sensor_idx)}

for i, sensor in enumerate(sensor_idx):
    plt.plot(time_points, h_residuals[i], label=f"Sensor {sensor_labels[sensor]}")

plt.axhline(0, color='black', linestyle='--', alpha=0.7)
plt.xlabel("Time (hours)")
plt.ylabel("Pressure Residuals (m)")
plt.title("Pressure Residuals Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Creating interactive charts
fig = go.Figure()

# reflex
sensor_labels = {sensor: f"Sensor {idx+1}" for idx, sensor in enumerate(sensor_idx)}

for i, sensor in enumerate(sensor_idx):
    fig.add_trace(go.Scatter(
        x=np.arange(h_0.shape[1]),
        y=h_residuals[i],
        mode='lines',
        name=sensor_labels[sensor],  # legend lable
        hoverinfo='x+y'  # Hovering the mouse will display the time and error value
    ))

# Setting the chart title and axis labels
fig.update_layout(
    title="Pressure Residuals Over Time",
    xaxis_title="Time (hours)",
    yaxis_title="Pressure Residuals (m)",
    legend_title="Sensor Number",
    template="plotly_white"
)

fig.show()

**<u>Question 1:</u>** Is the current hydraulic model of EXNING accurate according to hydraulic model calibration guidelines? Comment on the results of the preliminary model evaluation and, in particular, on
* the **sign** (i.e. are heads over or underestimated in the hydraulic simulation?) of pressure/head residuals,
* the **temporal** distribution (i.e. are the residuals time/flow dependent?) of pressure/head residuals,
* and the **spatial** distribution (i.e. are all sensors affected?) of the pressure/head residuals.

Given the information you were provided about the EXNING system and the results of the preliminary model evaluation, identify the most likely sources of model errors.

<font color="red">Enter response here...

### Part 1: Hydraulic model calibration (without regularisation)

Following initial reports concerning discrepancies in the recently calibrated EXNING model, AW were able to confirm that:
- flow and head sensors had been calibrated before collection of load/field data corresponding to the train and test datasets,
- demands in the train and test dataset are accurate (incl. for large users).

As a result, the remaining deviations between measured and simulated pressures must result from inaccurate model parameters. Errors associated with unknown valve status (for instance, unregistered closed valves) can be identified by solving a model calibration problem where pipe roughness coefficients are known, but valve minor/local loss coefficients are free to vary. In part 1, you will solve a hydraulic model calibration problem without regularisation. Complete the code below to calibrate the network model using the head measurements provided in data.

#### Split the data into *train* and *test* datasets.
We suggest using the first day worth of data as a <u>train dataset</u> and the remaining 3 days as a <u>test dataset</u>.

In [ ]:
# tain data
nt_train = 24
data_train = {
    'd': d_data[:, :nt_train],
    'h0': h0_data[:, :nt_train],
    'h_data': h_data[:, :nt_train]
}

# test data
data_test = {
    'd': d_data[:, nt_train:],
    'h0': h0_data[:, nt_train:],
    'h_data': h_data[:, nt_train:]
}

**<u>Question 2:</u>** Briefly comment on the definition of the train data set. What impact will it have on the generalizability of your model? (i.e., what range of conditions will you confidently be able to use your model for?)

#### Definition of the loss function.

In [ ]:
def loss_fun(h, h_data):
    return ( 1/len(h_data.flatten()) ) * cp.sum( ( h[sensor_idx, :] - h_data )**2 )

Compute MSE for C_0 values and using thetraining data.

In [ ]:
h_0 = h_0[:, :nt_train]
mse_0 = loss_fun(h_0, data_train['h_data']).value
mse_0

**<u>Question 3:</u>** Justify the choice/definition of the loss function, loss_fun.

#### Solve the parameter estimation problem using the train dataset
We first solve the hydraulic model calibration problem without regularisation. (You can reuse and adapt the code provided in Week 6.)

The following function is needed for the sequential convex programming (SCP) method used in this coursework. As with the `hydraulic_solver` function, we provide the following code in `linear_approx_calibration` for you to use throughout this notebook.

In [ ]:
#### DO NOT CHANGE THIS ####
# compute matrices for the head loss linear approximation of the HW headloss
def linear_approx_calibration(wdn, q, C):
    # unload data
    A12 = wdn.A12
    A10 = wdn.A10
    net_info = wdn.net_info
    link_df = wdn.link_df

    K = np.zeros((net_info['np'], 1))
    n_exp = link_df['n_exp'].astype(float).to_numpy().reshape(-1, 1)
    b1_k = copy.copy(K)
    b2_k = copy.copy(K)

    for idx, row in link_df.iterrows():
        if row['link_type'] == 'pipe':
            K[idx] = 10.67 * row['length'] * (C[idx] ** -row['n_exp']) * (row['diameter'] ** -4.8704)
            b1_k[idx] = copy.copy(K[idx])
            b2_k[idx] = (-n_exp[idx] * K[idx]) / C[idx]

        elif row['link_type'] == 'valve':
            K[idx] = (8 / (np.pi ** 2 * 9.81)) * (row['diameter'] ** -4) * C[idx]
            b1_k[idx] = -n_exp[idx] * copy.copy(K[idx])
            b2_k[idx] = copy.copy(K[idx]) / C[idx]

    a11_k = np.tile(K, q.shape[1]) * np.abs(q) ** (n_exp - 1)
    b1_k = np.tile(b1_k, q.shape[1]) * np.abs(q) ** (n_exp - 1)
    b2_k = np.tile(b2_k, q.shape[1]) * np.abs(q) ** (n_exp - 1) * q

    return a11_k, b1_k, b2_k

In [ ]:
# unload network training data
n_exp = link_df['n_exp']
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define problem parameters
Ki = np.inf
iter_max = 50
delta_k = 167
C_up_pipe = 200
C_lo_pipe = 1e-4

# initialise values
theta_k = C_0
q_k, h_k = hydraulic_solver(os.path.join(net_dir, net_name), d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
objval_k = loss_fun(h_k, h_data).value

### main scp code ###
for k in range(iter_max):

    # decision variables
    q = cp.Variable((net_info['np'], nt_train))
    h = cp.Variable((net_info['nn'], nt_train))
    theta = cp.Variable(net_info['np'])

    # objective function (defined using CVXPY functions)
    loss = ( 1/len(h_data.flatten()) ) * cp.sum_squares( h[sensor_idx,:] - h_data )
    objective = cp.Minimize(loss)

    # hydraulic feasibility constraints
    constraints = []
    for t in range(nt_train):
        # energy and mass constraints
        constraints += [
            cp.multiply(b1_k[:, t], q_k[:, t]) + cp.multiply(cp.multiply(n_exp, a11_k[:, t]), q[:, t]) + cp.multiply(b2_k[:, t], theta) + A12 @ h[:, t] + A10 @ h0[:, t] == 0,
            A12.T @ q[:, t] == d[:, t]
        ]

    # trust region constraints
    constraints += [
        cp.norm(theta - theta_k, 'inf') <= delta_k
    ]

    # variable bounds
    lower_bound_pipe = [theta[idx] >= C_lo_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += lower_bound_pipe
    upper_bound_pipe = [theta[idx] <= C_up_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += upper_bound_pipe

    # valve minor loss coefficient constraints
    valve_constraints = [theta[idx] == C_0[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'pipe']
    constraints += valve_constraints

    # solve optimisation problem
    problem = cp.Problem(objective, constraints)
    cvx_val = problem.solve(solver=cp.SCS, verbose=False, max_iters = 1000, eps=1e-3)

    # store optimal solution and compute actual and predicted decrease in objval
    theta_tilde = theta.value
    [q_tilde, h_tilde] = hydraulic_solver(os.path.join(net_dir, net_name), d, h0, C=theta_tilde)
    objval_tilde = loss_fun(h_tilde, h_data).value

    predicted_decrease = objval_k - cvx_val
    actual_decrease = objval_k - objval_tilde

    # evaluate latest SCP iteration
    if actual_decrease / predicted_decrease >= 0.1:
        objval_old = objval_k
        theta_k = theta_tilde
        q_k = q_tilde
        h_k = h_tilde
        objval_k = objval_tilde
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
        Ki = np.abs(objval_old - objval_k) / np.abs(objval_old)
        delta_k = 1.1 * delta_k
        print(f"Iteration {k} successful! Update estimate and increase trust region size. \n")

    else:
        delta_k = 0.25 * delta_k
        print(f"Iteration {k} unsuccessful! Return to previous estimate and reduce trust region size. \n")

    print(f"{k} {objval_k} {Ki} {delta_k} \n")

    if Ki <= 1e-3 or np.abs(objval_k) <= 1e-2 or delta_k <= 1e-1:
        break

delta167min

SCP implementation

**<u>Question 4:</u>** Describe the nature/role of the different outputs of the SCP algorithm printed above (objvalk, Ki, Deltak) and explain their trends.

Store the solution (i.e. new coefficients `theta_k`) as $C_1$.

In [ ]:
C_1 = theta_k
print(C_1)
q_k, h_k = hydraulic_solver(wdn, d, h0, C=C_1)
mes_train = np.mean((h_k[sensor_idx, :] - h_data) ** 2)
print(mes_train)

q_test, h_test = hydraulic_solver(wdn, data_test['d'], data_test['h0'], C=C_1)
mse_test = np.mean((h_test[sensor_idx,: ] - data_test['h_data']) ** 2)
print(f"\n测试集均方误差 (MSE): {mse_test:.6f}")

Evaluate final model error and visualise head residuals corresponding to *test* dataset.

In [ ]:

h_residuals_1 =  h_test[sensor_idx,:] - data_test['h_data']

print("Residuals shape:", h_residuals_1.shape)

In [ ]:
mean_residuals_1 = np.mean(h_residuals_1, axis=1)
plot_network(wdn, sensor_idx, vals=mean_residuals_1)

In [ ]:
plt.figure(figsize=(12, 6))
time_points = np.arange(data_test['h_data'].shape[1])  # Create a time index

sensor_labels = {sensor: str(idx+1) for idx, sensor in enumerate(sensor_idx)}

for i, sensor in enumerate(sensor_idx):
    plt.plot(time_points, h_residuals_1[i], label=f"Sensor {sensor_labels[sensor]}")

plt.axhline(0, color='black', linestyle='--', alpha=0.7)
plt.xlabel("Time (hours)")
plt.ylabel("Pressure Residuals (m)")
plt.title("Pressure Residuals Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
...

**<u>Question 5:</u>** Comment on the improvement in model accuracy (before vs. after calibration).

#### Discuss the values of the calibrated coefficients $C_1$
Visualise the values of newly calibrated coefficients $C_1$ compared to original model coefficients C.

In [ ]:
# C_1 value scatter plot (valves only)
valve_C1 = [C_1[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
fig, ax = plt.subplots()
ax.scatter(valve_C, valve_C1, facecolors='none', edgecolors='b',linewidths=0.75)
ax.set_xlabel('C_0 value', fontsize=12)
ax.set_ylabel('Calibrated C_1 value', fontsize=12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)

**<u>Question 6:</u>** Comment on the values of parameter estimates in and explain the results of the calibration without regularisation.

### Part 2: Hydraulic model calibration (with regularisation)

In order to reduce the underdeterminedness of the hydraulic model calibration problem (and improve the accuracy of the calibrated model), regularisation can be applied to incorporate prior knowledge about the expected variance or sparsity pattern of parameters $\theta$. Use the code provided in Weeks 5 and 6, and modify as necessary below to calibrate the hydraulic model with regularisation, using the same train data as before.

#### Definition of the regularisation function
Ridge regression (l2-regularisation) shrinks parameter estimates (without actually driving them to 0) in the hopes of reducing variance and improving prediction accuracy while lasso regression (-regularisation) encourages sparsity, driving many parameter estimates exactly to zero.

**<u>Question 7:</u>** Based on your answer to Question 6, comment on the expected variance or sparsity pattern of $\theta$. What type of regularisation (ridge or lasso) term should be included in the objective function of the model calibration problem?

The loss function is defined as:

In [ ]:
def reg_fun(h, h_data):
    return ( 1/len(h_data.flatten()) ) * cp.sum( ( h[sensor_idx, :] - h_data )**2 ) + lambda_0 * cp.norm(theta , 1)

#### Solve the parameter estimation problem using the train dataset
Solve the hydraulic model calibration problem with regularisation.

In [ ]:
lambda_0 = 0.1

In [ ]:
# unload network training data
n_exp = link_df['n_exp']
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define problem parameters
Ki = np.inf
iter_max = 50
delta_k = 400
C_up_pipe = 200
C_lo_pipe = 1e-4

# initialise values
theta_k = C_0
q_k, h_k = hydraulic_solver(os.path.join(net_dir, net_name), d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
objval_k = loss_fun(h_k, h_data).value

### main scp code ###
for k in range(iter_max):

    # decision variables
    q = cp.Variable((net_info['np'], nt_train))
    h = cp.Variable((net_info['nn'], nt_train))
    theta = cp.Variable(net_info['np'])

    # objective function (defined using CVXPY functions)
    loss = ( 1/len(h_data.flatten()) ) * cp.sum_squares( h[sensor_idx,:] - h_data )
    objective = cp.Minimize(loss + lambda_0 * cp.norm((theta - theta_k) , 1))

    # hydraulic feasibility constraints
    constraints = []
    for t in range(nt_train):
        # energy and mass constraints
        constraints += [
            cp.multiply(b1_k[:, t], q_k[:, t]) + cp.multiply(cp.multiply(n_exp, a11_k[:, t]), q[:, t]) + cp.multiply(b2_k[:, t], theta) + A12 @ h[:, t] + A10 @ h0[:, t] == 0,
            A12.T @ q[:, t] == d[:, t]
        ]

    # trust region constraints
    constraints += [
        cp.norm(theta - theta_k, 'inf') <= delta_k
    ]

    # variable bounds
    lower_bound_pipe = [theta[idx] >= C_lo_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += lower_bound_pipe
    upper_bound_pipe = [theta[idx] <= C_up_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += upper_bound_pipe

    # valve minor loss coefficient constraints
    valve_constraints = [theta[idx] == C_0[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'pipe']
    constraints += valve_constraints

    # solve optimisation problem
    problem = cp.Problem(objective, constraints)
    cvx_val = problem.solve(solver=cp.SCS, verbose=False, max_iters = 1000, eps=1e-3)

    # store optimal solution and compute actual and predicted decrease in objval
    theta_tilde = theta.value
    [q_tilde, h_tilde] = hydraulic_solver(os.path.join(net_dir, net_name), d, h0, C=theta_tilde)
    objval_tilde = loss_fun(h_tilde, h_data).value

    predicted_decrease = objval_k - cvx_val
    actual_decrease = objval_k - objval_tilde

    # evaluate latest SCP iteration
    if actual_decrease / predicted_decrease >= 0.1:
        objval_old = objval_k
        theta_k = theta_tilde
        q_k = q_tilde
        h_k = h_tilde
        objval_k = objval_tilde
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
        Ki = np.abs(objval_old - objval_k) / np.abs(objval_old)
        delta_k = 1.1 * delta_k
        print(f"Iteration {k} successful! Update estimate and increase trust region size. \n")

    else:
        delta_k = 0.25 * delta_k
        print(f"Iteration {k} unsuccessful! Return to previous estimate and reduce trust region size. \n")

    print(f"{k} {objval_k} {Ki} {delta_k} \n")

    if Ki <= 1e-3 or np.abs(objval_k) <= 1e-2 or delta_k <= 1e-1:
        break

Store the solution (i.e. new coefficients `theta_k`) as $C_2$.

2.065e-02

In [ ]:
C_2 = theta_k
print(C_2)
q_2, h_2 = hydraulic_solver(wdn, d, h0, C=C_2)
fk_2 = np.mean((h_2[sensor_idx, :] - h_data) ** 2)
print(fk_2)

q_test_2, h_test_2 = hydraulic_solver(wdn, data_test['d'], data_test['h0'], C=C_2)
mse_test_2 = np.mean((h_test_2[sensor_idx,: ] - data_test['h_data']) ** 2)
print(f"\n测试集均方误差 (MSE): {mse_test_2:.6f}")

In [ ]:
h_residuals_2 =  data_test['h_data'] - h_test_2[sensor_idx,:]

print("Residuals shape:", h_residuals_2.shape)
mean_residuals_2 = np.mean(h_residuals_2, axis=1)
plot_network(wdn, sensor_idx, vals=mean_residuals_2)

Evaluate final model error and visualise head residuals corresponding to *test* dataset.

In [ ]:
plt.figure(figsize=(12, 6))
time_points = np.arange(data_test['h_data'].shape[1])  # Create a time index

sensor_labels = {sensor: str(idx+1) for idx, sensor in enumerate(sensor_idx)}

for i, sensor in enumerate(sensor_idx):
    plt.plot(time_points, h_residuals_2[i], label=f"Sensor {sensor_labels[sensor]}")

plt.axhline(0, color='black', linestyle='--', alpha=0.7)
plt.xlabel("Time (hours)")
plt.ylabel("Pressure Residuals (m)")
plt.title("Pressure Residuals Over Time")
plt.legend()
plt.grid()
plt.show()

...

In [ ]:
# unload network training data
n_exp = link_df['n_exp'].to_numpy().reshape(-1, 1)
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define SCP problem parameters
Ki = np.inf
iter_max = 50
delta_k = 90
C_up_pipe = 200
C_lo_pipe = 1e-4

#initialise values
theta_k = C_0
q_k, h_k = hydraulic_solver(wdn, d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
objval_k = loss_fun(h_k, h_data).value
N = np.diag(n_exp.flatten())
### main scp code ###
for k in range(iter_max):
    # 构造凸优化问题
    theta = cp.Variable(len(theta_k))
    h_expr = cp.Variable((h_k.shape[0],h_k.shape[1]))  # Variable for head
    q_expr = cp.Variable((q_k.shape[0],q_k.shape[1]))  # Variable for flow
    objective = loss_fun(h_expr, h_data) + lambda_0 * cp.norm(theta_k - theta  , 1)


    constraints = []

    for t in range(24):
        constraints += [
        # energy and mass constraints

            cp.diag(b1_k[:,i]) @ q_k[:,i] + N @ cp.diag(a11_k[:,i]) @ q_expr[:,i] + cp.diag(b2_k[:,i]) @ theta + A12 @ h_expr[:,i] + A10 @ h0[:,i] == 0,
            # 流量平衡约束
            A21 @ q_expr[:,i] == d[:,i],

    ]
    # variable bounds
    lower_bound_pipe = [theta[idx] >= C_lo_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += lower_bound_pipe
    upper_bound_pipe = [theta[idx] <= C_up_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += upper_bound_pipe

    # valve minor loss coefficient constraints
    valve_constraints = [theta[idx] == C_0[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'pipe']
    constraints += valve_constraints
    constraints +=[
            # 信任域约束
        cp.norm( theta - theta_k , 'inf') <= delta_k
    ]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    cvx_val = problem.solve(solver=cp.SCS, eps=1e-3 ,max_iters=1000, verbose=False)

    # 获取候选解
    theta_candidate = theta.value
    if theta_candidate is None:
        delta_k *= 0.2
        continue
    [q_candidate,  h_candidate] =   hydraulic_solver(wdn, d, h0, C=theta_candidate)
    objval_candidate = loss_fun(h_candidate, h_data).value

    denominator = objval_k - problem.value
    if abs(denominator) < 1e-10:
       rho = 0  # 视为无改进
    else:
       rho = (objval_k - objval_candidate) / denominator


    # 信任域调整逻辑
    if rho >= 0.1:
        # 接受新解
        Ki = np.abs(objval_k - objval_candidate) / abs(objval_k)
        theta_k = theta_candidate.copy()
        q_k = q_candidate.copy()
        h_k = h_candidate.copy()
        objval_k = objval_candidate
        delta_k = 1.1 * delta_k
        print(f"Iteration {k} successful! Update estimate and increase trust region size. \n")
        # 更新线性化矩阵
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
    else:
        # 拒绝新解
        delta_k = 0.25 * delta_k
        print(f"Iteration {k} unsuccessful!{objval_k} Return to previous estimate and reduce trust region size{delta_k}. \n")

    if  Ki<=1e-3 or delta_k <= 1e-4:
        print(f"第{k+1}次完成迭代")
        break

Iteration 0 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size22.5. 

Iteration 1 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size5.625. 

Iteration 2 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size1.40625. 

Iteration 3 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size0.3515625. 

Iteration 4 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size0.087890625. 

Iteration 5 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size0.02197265625. 

Iteration 6 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size0.0054931640625. 

Iteration 7 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region size0.001373291015625. 

Iteration 8 unsuccessful!9.705414431633923 Return to previous estimate and reduce trust region 

**<u>Question 8:</u>** Comment on the improvement in model accuracy after calibration with regularisation.

#### Discuss the values of the calibrated coefficients $C_2$
Visualise the values of newly calibrated coefficients $C_2$.

In [ ]:
...

**<u>Question 9:</u>** Comment on the values of parameter estimates in $C_2$ compared to $C_1$ and explain the results of the calibration with regularisation.

**<u>Question 10:</u>** Do the calibrated local loss coefficient values in $C_2$ corroborate the conclusions of the preliminary analysis about the most likely sources of error in the EXNING model (existence/location of unknowingly closed valves)? Summarise your findings (100-150 words + 1-2 figures) and provide recommendations to AW to validate your proposed hydraulic model update.


In [ ]:
# unload network training data
n_exp = link_df['n_exp'].to_numpy().reshape(-1, 1)
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define SCP problem parameters
Ki = np.inf
iter_max = 50
delta_k = 120
C_up_pipe = 200
C_lo_pipe = 1e-4

#initialise values
theta_k = C_0
q_k, h_k = hydraulic_solver(wdn, d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
objval_k = loss_fun(h_k, h_data).value
N = np.diag(n_exp.flatten())
### main scp code ###
for k in range(iter_max):
    # 构造凸优化问题
    theta = cp.Variable(len(theta_k))
    h = cp.Variable((h_k.shape[0],h_k.shape[1]))  # Variable for head
    q = cp.Variable((q_k.shape[0],q_k.shape[1]))  # Variable for flow
    objective = loss_fun(h, h_data)


    constraints = []

    for i in range(24):
        constraints += [
        # energy and mass constraints

             cp.diag(b1_k[:,i]) @ q_k[:,i] + N @ cp.diag(a11_k[:,i]) @ q[:,i] + cp.diag(b2_k[:,i]) @ theta + A12 @ h[:,i] + A10 @ h0[:,i] == 0,
             #流量平衡约束
            A21 @ q[:,i] == d[:,i],

    ]
    #for t in range(24):
        # energy and mass constraints
        #constraints += [
            #cp.multiply(cp.diag(b1_k[:, t]), q_k[:, t]) + cp.multiply(cp.multiply(cp.diag(n_exp), np.diag(a11_k[:, t])), q[:, t]) + cp.multiply(cp.diag(b2_k[:, t]), theta) + A12 @ h[:, t] + A10 @ h0[:, t] == 0,
            #A12.T @ q[:, t] == d[:, t]
      #  ]
    # variable bounds
    lower_bound_pipe = [theta[idx] >= C_lo_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += lower_bound_pipe
    upper_bound_pipe = [theta[idx] <= C_up_pipe for idx, row in link_df.iterrows() if row['link_type'] == 'valve']
    constraints += upper_bound_pipe

    # valve minor loss coefficient constraints
    valve_constraints = [theta[idx] == C_0[idx] for idx, row in link_df.iterrows() if row['link_type'] == 'pipe']
    constraints += valve_constraints
    constraints +=[
            # 信任域约束
        cp.norm( theta - theta_k , 'inf') <= delta_k
    ]
    problem = cp.Problem(cp.Minimize(objective), constraints)
    cvx_val = problem.solve(solver=cp.SCS ,max_iters=1000, verbose=False)

    # 获取候选解
    theta_candidate = theta.value
    if theta_candidate is None:
        delta_k *= 0.2
        continue
    [q_candidate,  h_candidate] =   hydraulic_solver(wdn, d, h0, C=theta_candidate)
    objval_candidate = loss_fun(h_candidate, h_data).value

    denominator = objval_k - problem.value
    if abs(denominator) < 1e-10:
       rho = 0  # 视为无改进
    else:
       rho = (objval_k - objval_candidate) / denominator


    # 信任域调整逻辑
    if rho >= 0.1:
        # 接受新解
        Ki = np.abs(objval_k - objval_candidate) / abs(objval_k)
        theta_k = theta_candidate.copy()
        q_k = q_candidate.copy()
        h_k = h_candidate.copy()
        objval_k = objval_candidate
        delta_k = 1.1 * delta_k
        print(f"Iteration {k} successful! Update estimate and increase trust region size. \n")
        # 更新线性化矩阵
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
    else:
        # 拒绝新解
        delta_k = 0.25 * delta_k
        print(f"Iteration {k} unsuccessful!{objval_k} Return to previous estimate and reduce trust region size{delta_k}. \n")

    if  Ki<=1e-3 or delta_k <= 1e-3:
        print(f"第{k+1}次完成迭代")
        break


In [ ]:
# unload network training data
n_exp = link_df['n_exp'].to_numpy().reshape(-1, 1)
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define SCP problem parameters
Ki = np.inf
iter_max = 50
delta_k = 20
C_up_pipe = 200
C_lo_pipe = 1e-03
CHECK=0
#initialise values
theta_k = link_df['C'].to_numpy()
q_k, h_k = hydraulic_solver(wdn, d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
fk = np.mean((h_k[sensor_idx, :] - h_data) ** 2)
objval_k = loss_fun(h_k, h_data).value
### main scp code ###
for k in range(iter_max):
    # 构造凸优化问题
    theta = cp.Variable(len(theta_k))
    h_expr = cp.Variable((h_k.shape[0],h_k.shape[1]))  # Variable for head
    q_expr = cp.Variable((q_k.shape[0],q_k.shape[1]))  # Variable for flow
    objective = loss_fun(h_expr, h_data) + lambda_0 * cp.norm(theta , 1)


    constraints = []

    for i in range(24):
        constraints += [
        # 线性化后的能量守恒方程

            cp.diag(b1_k[:,i]) @ q_k[:,i] + N @ cp.diag(a11_k[:,i]) @ q_expr[:,i] + cp.diag(b2_k[:,i]) @ theta + A12 @ h_expr[:,i] + A10 @ h0[:,i] == 0,
            # 流量平衡约束
            A21 @ q_expr[:,i] == d[:,i],

    ]
    constraints +=[
         # 参数上下限
        theta >= C_lo_pipe,
        theta <= C_up_pipe,
            # 信任域约束
        cp.norm( theta - theta_k , 'inf') <= delta_k
    ]
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve(solver = cp.SCS,eps=1e-6,max_iters=5000, verbose=False)

    # 获取候选解
    theta_candidate = theta.value
    if theta_candidate is None:
        delta_k *= 0.5
        continue
    q_candidate,  h_candidate =   hydraulic_solver(wdn, d, h0, C=theta_candidate)
    objval_candidate = loss_fun(h_candidate, h_data).value + lambda_0 * cp.norm(theta_candidate , 1).value

    denominator = objval_k - prob.value
    if abs(denominator) < 1e-10:
      rho = 0  # 视为无改进
    else:
       rho = (objval_k - objval_candidate) / denominator


    # 信任域调整逻辑
    if rho >= 0.01:
        # 接受新解
        Ki = abs(objval_k - objval_candidate) / abs(objval_k)
        theta_k = theta_candidate.copy()
        q_k = q_candidate.copy()
        h_k = h_candidate.copy()
        objval_k = loss_fun(h_k, h_data).value + lambda_0 * cp.norm(theta_k , 1).value
        delta_k = 1.1 * delta_k

        # 更新线性化矩阵
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
    else:
        # 拒绝新解
        delta_k = 0.5 * delta_k


    if  Ki<=1e-3 or delta_k <= 1e-4:
        print(f"第{k+1}次完成迭代")
        break


Failure:interrupted


SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
# unload network training data
n_exp = link_df['n_exp'].to_numpy().reshape(-1, 1)
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define SCP problem parameters
Ki = np.inf
iter_max = 50
delta_k = 25
C_up_pipe = 200
C_lo_pipe = 1.1e-4
CHECK=0
#initialise values
theta_k = link_df['C'].to_numpy()
q_k, h_k = hydraulic_solver(wdn, d, h0, C=theta_k)
a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
fk = np.mean((h_k[sensor_idx, :] - h_data) ** 2)
objval_k = loss_fun(h_k, h_data).value
N = np.diag(n_exp.flatten())
### main scp code ###
for k in range(iter_max):
    # 构造凸优化问题
    theta = cp.Variable(len(theta_k))
    h_expr = cp.Variable((h_k.shape[0],h_k.shape[1]))  # Variable for head
    q_expr = cp.Variable((q_k.shape[0],q_k.shape[1]))  # Variable for flow
    objective = loss_fun(h_expr, h_data)


    constraints = []

    for i in range(24):
        constraints += [
        # 线性化后的能量守恒方程

            cp.diag(b1_k[:,i]) @ q_k[:,i] + N @ cp.diag(a11_k[:,i]) @ q_expr[:,i] + cp.diag(b2_k[:,i]) @ theta + A12 @ h_expr[:,i] + A10 @ h0[:,i] == 0,
            # 流量平衡约束
            A21 @ q_expr[:,i] == d[:,i],

    ]
    constraints +=[
         # 参数上下限
        theta >= C_lo_pipe,
        theta <= C_up_pipe,
            # 信任域约束
        cp.norm( theta - theta_k , 'inf') <= delta_k
    ]
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve(solver = cp.SCS,eps=1e-6,max_iters=5000, verbose=False)

    # 获取候选解
    theta_candidate = theta.value
    if theta_candidate is None:
        delta_k *= 0.5
        continue
    q_candidate,  h_candidate =   hydraulic_solver(wdn, d, h0, C=theta_candidate)
    objval_candidate = loss_fun(h_candidate, h_data).value

    denominator = objval_k - prob.value
    if abs(denominator) < 1e-10:
      rho = 0  # 视为无改进
    else:
       rho = (objval_k - objval_candidate) / denominator


    # 信任域调整逻辑
    if rho >= 0.01:
        # 接受新解
        Ki = abs(objval_k - objval_candidate) / abs(objval_k)
        theta_k = theta_candidate.copy()
        q_k = q_candidate.copy()
        h_k = h_candidate.copy()
        objval_k = loss_fun(h_k, h_data).value
        delta_k = 1.2 * delta_k

        # 更新线性化矩阵
        a11_k, b1_k, b2_k = linear_approx_calibration(wdn, q_k, theta_k)
    else:
        # 拒绝新解
        delta_k = 0.6 * delta_k


    if  Ki<=1e-3 or delta_k <= 1e-4:
        print(f"第{k+1}次完成迭代")
        break
